# Lab: NBA Players Analyse

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# 1.Data process and time series plot

1.Load the data

games_details.csv includes details of games dataset, all statistics of players for a given game



In [ ]:
df=pd.read_csv('games_details.csv',header=0,low_memory=False)
print(df)

2.Preprocess the data

fill none with "00:00" in the column MIN

fill none with 0 in the column FGM to the last column

modify the MIN column to make sure the time is correct

In [ ]:
df[df.columns[10:]].fillna(0,inplace=True)
df['MIN'].fillna('00:00',inplace=True)
def deal_time(x):
    split_string=x.split(':')
    return float(split_string[0])
df['MIN']=df['MIN'].apply(lambda x:deal_time(x))
print(df['MIN'])


3.Extract the star players data 



In [ ]:
group=['LeBron James','Kevin Durant']
dic_group={}
for name in group:
    dic_group[name]=df[df['PLAYER_NAME'].isin([name])].copy()
    dic_group[name].reset_index(inplace=True)
print(dic_group.keys())


4.Get the game date for every player from  games.csv according to the game id

In [ ]:
df_games=pd.read_csv('games.csv',header=0,low_memory=False)
print(df_games)
df_games['GAME_ID']=df_games['GAME_ID'].astype(int)

5.Get the game date for every player

In [ ]:
for key in dic_group.keys():
    df_date=df_games[df_games['GAME_ID'].isin(dic_group[key]['GAME_ID'])].copy()
    
    df_date.drop_duplicates('GAME_ID',keep='first',inplace=True)
    df_date.set_index('GAME_ID',inplace=True)
    print(key)
    dic_group[key]['GAME_ID']=dic_group[key]['GAME_ID'].astype(int)
   
    for row in range(dic_group[key].shape[0]):
        if dic_group[key].loc[row,'GAME_ID'] in df_date.index:
            dic_group[key].loc[row,'GAME_DATE_EST']=df_date.loc[dic_group[key].loc[row,'GAME_ID'],'GAME_DATE_EST']
    dic_group[key].dropna(subset=['GAME_DATE_EST'],inplace=True)
    dic_group[key].set_index('GAME_DATE_EST',inplace=True)
    dic_group[key].sort_index(ascending=True,inplace=True)#reverse the sequence
   # dic_group1[key].index=np.arange(dic_group1[key].shape[0])
print(dic_group[group[0]][['GAME_ID']])



6.Plot time series

In [ ]:
internal=200
for key in dic_group.keys():
    y_pts=dic_group[key]['PTS']
    y_fgm=dic_group[key]['FGM']
    y_fg3m=dic_group[key]['FG3M']
    y_time=dic_group[key]['MIN']
    x_ticks=np.arange(dic_group[key].shape[0])
    num=int(dic_group[key].shape[0]/internal)
    new_xticks=np.arange(num)  
    new_xticks=[x*internal for x in new_xticks]
    new_xticks.append(dic_group[key].shape[0]-50)
    xlabels=[dic_group[key].index[x] for x in new_xticks]
    plt.subplots(figsize=(16, 6))
    plt.title(f'{key} PTS\FGM\FG3M\MIN  trend')
    plt.ylabel('number')
    plt.xticks(new_xticks,xlabels,rotation=45,fontsize=8) 
    plt.plot(y_pts,color='b',label='PTS')
    plt.plot(y_fgm,color='y',label='FGM')
    plt.plot(y_fg3m,color='g',label='FG3M')
    plt.plot(y_time,color='r',label='MIN')
    plt.legend()
    plt.show()


    

These stars show cyclical changes in playing time and the performance of major offensive and defensive indicators, even at the end of their careers, the relevant data did not appear to be significantly worse in one direction

# 2. Stationarity Tests

1.Calculate the rolling mean and standard deviation of the input time series.



The size of the rolling window is governed by the argument `wd_size`.

In [ ]:
# calculate the weighted moving average

def calc_wma(ser, wd_size, weights=1):
    if isinstance(weights, int):
        weights = np.full(wd_size, weights, dtype=float)

    wma = []
    for i in range(len(ser)):
        low, high = max(0, i - wd_size + 1), i + 1
        wma.append(np.average(ser.iloc[low: high], weights=weights[-(high - low):]))
    return np.array(wma)

calculate the rolling statistics,use PTS and BLK respectively

In [ ]:

wd_size = 7 # our window size, k
dic_group1_rolling_data={}
dic_group2_rolling_data={}
for key in dic_group.keys():
    first_mmt = calc_wma(dic_group[key]['PTS'], wd_size)
    second_mmt = calc_wma(dic_group[key]['PTS'] ** 2, wd_size)
    rolling_data=[]
    rolling_mean = first_mmt
    rolling_std = np.sqrt(second_mmt - first_mmt ** 2)
    rolling_data.append(rolling_mean)
    rolling_data.append(rolling_std)
    dic_group1_rolling_data[key]=rolling_data
print(dic_group1_rolling_data)




2.plot the time series and the rolling statistics

In [ ]:


for key in dic_group.keys():
    y_pts=dic_group[key]['PTS']
    y_mean=dic_group1_rolling_data[key][0]
    y_std=dic_group1_rolling_data[key][1]
    x_ticks=np.arange(dic_group[key].shape[0])
    num=int(dic_group[key].shape[0]/internal)
    new_xticks=np.arange(num)  
    new_xticks=[x*internal for x in new_xticks]
    new_xticks.append(dic_group[key].shape[0]-50)
    xlabels=[dic_group[key].index[x] for x in new_xticks]
    plt.subplots(figsize=(16, 6))
    plt.title(f"{key} PTS trend \n" + f"Rolling Stats with Window Size = {wd_size} ")
    plt.ylabel('number')
    plt.xticks(new_xticks,xlabels,rotation=45,fontsize=8) 
    plt.plot(y_pts,color='b',label='Original')
    plt.plot(y_mean,color='r',label='mean')
    plt.plot(y_std,color='g',label='std')
    plt.legend()
    plt.show()
    


3.set a window size of 3 

In [ ]:
wd_size = 3 # our window size, k
dic_group1_rolling_data={}
dic_group2_rolling_data={}
for key in dic_group.keys():
    first_mmt = calc_wma(dic_group[key]['PTS'], wd_size)
    second_mmt = calc_wma(dic_group[key]['PTS'] ** 2, wd_size)
    rolling_data=[]
    rolling_mean = first_mmt
    rolling_std = np.sqrt(second_mmt - first_mmt ** 2)
    rolling_data.append(rolling_mean)
    rolling_data.append(rolling_std)
    dic_group1_rolling_data[key]=rolling_data
print(dic_group1_rolling_data)


In [ ]:
for key in dic_group.keys():
    y_pts=dic_group[key]['PTS']
    y_mean=dic_group1_rolling_data[key][0]
    y_std=dic_group1_rolling_data[key][1]
    x_ticks=np.arange(dic_group[key].shape[0])
    num=int(dic_group[key].shape[0]/internal)
    new_xticks=np.arange(num)  
    new_xticks=[x*internal for x in new_xticks]
    new_xticks.append(dic_group[key].shape[0]-50)
    xlabels=[dic_group[key].index[x] for x in new_xticks]
    plt.subplots(figsize=(16, 6))
    plt.title(f"{key} PTS trend \n" + f"Rolling Stats with Window Size = {wd_size} ")
    plt.ylabel('number')
    plt.xticks(new_xticks,xlabels,rotation=45,fontsize=8) 
    plt.plot(y_pts,color='b',label='Original')
    plt.plot(y_mean,color='r',label='mean')
    plt.plot(y_std,color='g',label='std')
    plt.legend()
    plt.show()
    


4.set a window size of 10

In [ ]:
wd_size = 10 # our window size, k
dic_group1_rolling_data={}
dic_group2_rolling_data={}
for key in dic_group.keys():
    first_mmt = calc_wma(dic_group[key]['PTS'], wd_size)
    second_mmt = calc_wma(dic_group[key]['PTS'] ** 2, wd_size)
    rolling_data=[]
    rolling_mean = first_mmt
    rolling_std = np.sqrt(second_mmt - first_mmt ** 2)
    rolling_data.append(rolling_mean)
    rolling_data.append(rolling_std)
    dic_group1_rolling_data[key]=rolling_data
print(dic_group1_rolling_data)


In [ ]:
for key in dic_group.keys():
    y_pts=dic_group[key]['PTS']
    y_mean=dic_group1_rolling_data[key][0]
    y_std=dic_group1_rolling_data[key][1]
    x_ticks=np.arange(dic_group[key].shape[0])
    num=int(dic_group[key].shape[0]/internal)
    new_xticks=np.arange(num)  
    new_xticks=[x*internal for x in new_xticks]
    new_xticks.append(dic_group[key].shape[0]-50)
    xlabels=[dic_group[key].index[x] for x in new_xticks]
    plt.subplots(figsize=(16, 6))
    plt.title(f"{key} PTS trend \n" + f"Rolling Stats with Window Size = {wd_size} ")
    plt.ylabel('number')
    plt.xticks(new_xticks,xlabels,rotation=45,fontsize=8) 
    plt.plot(y_pts,color='b',label='Original')
    plt.plot(y_mean,color='r',label='mean')
    plt.plot(y_std,color='g',label='std')
    plt.legend()
    plt.show()
    


# Conclusion
As can be seen from Window size 3, 7, and 10, the trend of the mean and the original values remains consistent,at the same time, std deviation is kept in a small floating range,so time series above is stationary.
Comparing Window size 3, 7, and 10, it should be that the smaller the window, the better the performance of window size 3, indicating that the window size should not be too large.

# 3. Seasonal patterns

seasonal patterns analyse using PTS and BLK.

In [ ]:
for key in dic_group.keys():
    dic_group[key].dropna(subset=['PTS'],inplace=True)
    dic_group[key].reset_index(inplace=True)
    
    
    

In [ ]:
data_average=[]
for key in dic_group.keys():
    year_data=[]
    year_data_all=[]
    year_record=[]
    year1=''
    year2=''
    for row in range(dic_group[key].shape[0]-1):
        year1=dic_group[key].loc[row,'GAME_DATE_EST'].split('-')[0]
        year2=dic_group[key].loc[row+1,'GAME_DATE_EST'].split('-')[0]
        if year1==year2:
            year_data.append(dic_group[key].loc[row,'PTS'])
        else:
            year_record.append(year1)
            year_data.append(dic_group[key].loc[row,'PTS'])
            year_data_all.append(year_data.copy())
            year_data=[]
    year_data_all.append(year_data.copy())
    year_record.append(year1)
    print(key)
    plt.subplots(figsize=(20, 6))
    data_list=[]
    for index in range(len(year_data_all)):
        plt.plot(year_data_all[index],label=year_record[index]+' '+str(sum(year_data_all[index])))
        data_list.append(sum(year_data_all[index]))
    plt.title(f'{key} PTS every year')
    plt.legend()
    plt.show()
    
    plt.subplots(figsize=(20, 6))
    plt.title(f'{key} PTS trend every year')
    plt.plot(year_record,data_list)
    for index in range(len(year_record)):
        plt.text(index,data_list[index],data_list[index])
    plt.show()
    
    data_average.append(sum(data_list)/len(year_record))
plt.title(f'{list(dic_group.keys())[0]} VS {list(dic_group.keys())[1]} average annual PTS')
for index in range(len(dic_group.keys())):
    plt.bar(list(dic_group.keys())[index],data_average[index])
    plt.text(index,data_average[index],data_average[index])
plt.show()

In [ ]:
data_average=[]
for key in dic_group.keys():
    year_data=[]
    year_data_all=[]
    year_record=[]
    year1=''
    year2=''
    for row in range(dic_group[key].shape[0]-1):
        year1=dic_group[key].loc[row,'GAME_DATE_EST'].split('-')[0]
        year2=dic_group[key].loc[row+1,'GAME_DATE_EST'].split('-')[0]
        if year1==year2:
            year_data.append(dic_group[key].loc[row,'FGM'])
        else:
            year_record.append(year1)
            year_data.append(dic_group[key].loc[row,'FGM'])
            year_data_all.append(year_data.copy())
            year_data=[]
    year_data_all.append(year_data.copy())
    year_record.append(year1)
    print(key)
    plt.subplots(figsize=(20, 6))
    data_list=[]
    for index in range(len(year_data_all)):
        plt.plot(year_data_all[index],label=year_record[index]+' average '+str(round(sum(year_data_all[index])/len(year_data_all[index]),2)))
        data_list.append(round(sum(year_data_all[index])/len(year_data_all[index]),2))
    plt.title(f'{key} FGM every year')
    plt.legend()
    plt.show()
    
    plt.subplots(figsize=(20, 6))
    plt.title(f'{key} FGM trend every year')
    plt.plot(year_record,data_list)
    for index in range(len(year_record)):
        plt.text(index,data_list[index],data_list[index])
    plt.show()
    
    data_average.append(round(sum(data_list)/len(year_record),2))
plt.title(f'{list(dic_group.keys())[0]} VS {list(dic_group.keys())[1]} average annual FGM')
for index in range(len(dic_group.keys())):
    plt.bar(list(dic_group.keys())[index],data_average[index])
    plt.text(index,data_average[index],data_average[index])
plt.show()

In [ ]:
data_average=[]
for key in dic_group.keys():
    year_data=[]
    year_data_all=[]
    year_record=[]
    year1=''
    year2=''
    for row in range(dic_group[key].shape[0]-1):
        year1=dic_group[key].loc[row,'GAME_DATE_EST'].split('-')[0]
        year2=dic_group[key].loc[row+1,'GAME_DATE_EST'].split('-')[0]
        if year1==year2:
            year_data.append(dic_group[key].loc[row,'FG3M'])
        else:
            year_record.append(year1)
            year_data.append(dic_group[key].loc[row,'FG3M'])
            year_data_all.append(year_data.copy())
            year_data=[]
    year_data_all.append(year_data.copy())
    year_record.append(year1)
    print(key)
    plt.subplots(figsize=(20, 6))
    data_list=[]
    for index in range(len(year_data_all)):
        plt.plot(year_data_all[index],label=year_record[index]+' average '+str(round(sum(year_data_all[index])/len(year_data_all[index]),2)))
        data_list.append(round(sum(year_data_all[index])/len(year_data_all[index]),2))
    plt.title(f'{key} FG3M every year')
    plt.legend()
    plt.show()
    
    plt.subplots(figsize=(20, 6))
    plt.title(f'{key} FG3M trend every year')
    plt.plot(year_record,data_list)
    for index in range(len(year_record)):
        plt.text(index,data_list[index],data_list[index])
    plt.show()
    
    data_average.append(round(sum(data_list)/len(year_record),2))
plt.title(f'{list(dic_group.keys())[0]} VS {list(dic_group.keys())[1]} average annual FG3M')
for index in range(len(dic_group.keys())):
    plt.bar(list(dic_group.keys())[index],data_average[index])
    plt.text(index,data_average[index],data_average[index])
plt.show()